<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/32_ntb_PySpark_RenameCollumns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###  Renomeando de forma dinâmica as colunas com PySpark
 > Uso Dicionário para renomar


- Criado em 28.01.25

- Por: Luciana Sampaio
- Email: luciana.sampaio84@gmail.com
- LinkedIn: Luciana Sampaio

   - testando modularizar code
   

In [ ]:
#Example to load csv
import pandas as pd
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("CSVLoad").getOrCreate()

sdf = spark.createDataFrame(pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/airline-safety/airline-safety.csv"))

In [ ]:
import requests

def get_response(url):
    try:
        response = requests.get(url)
        return response
    except Exception as e:
        print(e)
        raise


In [ ]:
sdf.show()

+--------------------+----------------------+---------------+---------------------+----------------+---------------+---------------------+----------------+
|             airline|avail_seat_km_per_week|incidents_85_99|fatal_accidents_85_99|fatalities_85_99|incidents_00_14|fatal_accidents_00_14|fatalities_00_14|
+--------------------+----------------------+---------------+---------------------+----------------+---------------+---------------------+----------------+
|          Aer Lingus|             320906734|              2|                    0|               0|              0|                    0|               0|
|           Aeroflot*|            1197672318|             76|                   14|             128|              6|                    1|              88|
|Aerolineas Argent...|             385803648|              6|                    0|               0|              1|                    0|               0|
|         Aeromexico*|             596871813|              3|   

In [ ]:
# Dicionário de renomeação (coluna_atual -> novo_nome)
rename_dict = {
    "airline": "Companhia",
    "fatal_accidents_85_99": "acidentes_fatais_85_99",
    "incidents_00_14": "incidentes_00_14",
    "avail_seat_km_per_week": "assento_disponível_km_por_semana",
    "fatalities_00_14": "fatais_00_14"

}


# Aplicando renomeação usando loop
# Changed 'df' to 'sdf' to work with the Spark DataFrame
for old_col, new_col in rename_dict.items():
    sdf = sdf.withColumnRenamed(old_col, new_col)
# Assigning the final DataFrame to 'df' for consistency
df = sdf

In [ ]:
df.show(7)

+--------------------+--------------------------------+---------------+----------------------+----------------+----------------+---------------------+------------+
|           Companhia|assento_disponível_km_por_semana|incidents_85_99|acidentes_fatais_85_99|fatalities_85_99|incidentes_00_14|fatal_accidents_00_14|fatais_00_14|
+--------------------+--------------------------------+---------------+----------------------+----------------+----------------+---------------------+------------+
|          Aer Lingus|                       320906734|              2|                     0|               0|               0|                    0|           0|
|           Aeroflot*|                      1197672318|             76|                    14|             128|               6|                    1|          88|
|Aerolineas Argent...|                       385803648|              6|                     0|               0|               1|                    0|           0|
|         Aerome

In [ ]:
# Convertendo nomes das colunas para maiusculas
df = df.toDF(*[col.upper() for col in df.columns])

In [ ]:
df.show(7)

+--------------------+--------------------------------+---------------+----------------------+----------------+----------------+---------------------+------------+
|           COMPANHIA|ASSENTO_DISPONÍVEL_KM_POR_SEMANA|INCIDENTS_85_99|ACIDENTES_FATAIS_85_99|FATALITIES_85_99|INCIDENTES_00_14|FATAL_ACCIDENTS_00_14|FATAIS_00_14|
+--------------------+--------------------------------+---------------+----------------------+----------------+----------------+---------------------+------------+
|          Aer Lingus|                       320906734|              2|                     0|               0|               0|                    0|           0|
|           Aeroflot*|                      1197672318|             76|                    14|             128|               6|                    1|          88|
|Aerolineas Argent...|                       385803648|              6|                     0|               0|               1|                    0|           0|
|         Aerome

In [ ]:
# lista das empresas aerea

df.select("COMPANHIA").rdd.flatMap(lambda x: x).collect()


['Aer Lingus',
 'Aeroflot*',
 'Aerolineas Argentinas',
 'Aeromexico*',
 'Air Canada',
 'Air France',
 'Air India*',
 'Air New Zealand*',
 'Alaska Airlines*',
 'Alitalia',
 'All Nippon Airways',
 'American*',
 'Austrian Airlines',
 'Avianca',
 'British Airways*',
 'Cathay Pacific*',
 'China Airlines',
 'Condor',
 'COPA',
 'Delta / Northwest*',
 'Egyptair',
 'El Al',
 'Ethiopian Airlines',
 'Finnair',
 'Garuda Indonesia',
 'Gulf Air',
 'Hawaiian Airlines',
 'Iberia',
 'Japan Airlines',
 'Kenya Airways',
 'KLM*',
 'Korean Air',
 'LAN Airlines',
 'Lufthansa*',
 'Malaysia Airlines',
 'Pakistan International',
 'Philippine Airlines',
 'Qantas*',
 'Royal Air Maroc',
 'SAS*',
 'Saudi Arabian',
 'Singapore Airlines',
 'South African',
 'Southwest Airlines',
 'Sri Lankan / AirLanka',
 'SWISS*',
 'TACA',
 'TAM',
 'TAP - Air Portugal',
 'Thai Airways',
 'Turkish Airlines',
 'United / Continental*',
 'US Airways / America West*',
 'Vietnam Airlines',
 'Virgin Atlantic',
 'Xiamen Airlines']

In [ ]:
df.printSchema()

root
 |-- COMPANHIA: string (nullable = true)
 |-- ASSENTO_DISPONÍVEL_KM_POR_SEMANA: long (nullable = true)
 |-- INCIDENTS_85_99: long (nullable = true)
 |-- ACIDENTES_FATAIS_85_99: long (nullable = true)
 |-- FATALITIES_85_99: long (nullable = true)
 |-- INCIDENTES_00_14: long (nullable = true)
 |-- FATAL_ACCIDENTS_00_14: long (nullable = true)
 |-- FATAIS_00_14: long (nullable = true)

